# AIHub여행객정보 ➡️ user 테이블 매핑

## User ID Data Generation

In [20]:
# 데이터 전처리 및 변환 스크립트
# 이 코드는 원본 데이터셋을 우리 시스템에 맞게 변환하는 작업을 수행합니다.
# 1. 원본 데이터 파일 로드 (여행객 Master, 여행 데이터)
# 2. User 테이블 생성 - TRAVELER_ID에서 문자 제거 후 숫자만 추출하여 user_id 생성
# 3. Plan 테이블 생성 - Master와 Travel 데이터 조인 후 필요한 필드 추출
# 4. 생성된 데이터를 CSV 파일로 저장

import pandas as pd

# 데이터 파일 읽기
df_master = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/capital/tn_traveller_master_여행객 Master_E.csv', encoding='utf-8')
df_travel = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/capital/tn_travel_여행_E.csv', encoding='utf-8')

print("Master 데이터 컬럼:", df_master.columns.tolist())
print("Travel 데이터 컬럼:", df_travel.columns.tolist())
print(f"Master 데이터 길이: {len(df_master)}")
print(f"Travel 데이터 길이: {len(df_travel)}")

# 1. User 테이블 데이터 생성 
user_df = pd.DataFrame()

# 단계별로 변환하면서 확인
# 1단계: 문자 제거 후 결과 확인
temp_id = df_master['TRAVELER_ID'].str.replace(r'[a-zA-Z]', '', regex=True)
print("문자 제거 후:", temp_id.head())

# 2단계: 숫자만 남은 것을 정수로 변환
temp_id = temp_id.astype(int)
print("정수 변환 후:", temp_id.head())

# 3단계: 최종 계산 - user_id 생성 방식 통일
user_df['user_id'] = temp_id
user_df['email'] = df_master['TRAVELER_ID'].astype(str) + '@gmail.com'
user_df['name'] = df_master['TRAVELER_ID'] # 임시 이름

print("\nUser 테이블 최종 결과:")
print(user_df.head())
print(f"User 테이블 길이: {len(user_df)}")

# 2. Plan 테이블 데이터 생성
# Master와 Travel 데이터 조인
df_plan = pd.merge(df_travel, df_master[['TRAVELER_ID', 'TRAVEL_STATUS_DESTINATION']], 
                   on='TRAVELER_ID', how='left')
print(f"조인 후 Plan 데이터 길이: {len(df_plan)}")

# 새로운 plan_df 생성
plan_df = pd.DataFrame()

# plan_id 생성
plan_df['plan_id'] = range(1000, 1000 + len(df_plan))

# user_id 생성 - user 테이블과 동일한 방식 사용
traveler_ids = df_plan['TRAVELER_ID'].str.replace(r'[a-zA-Z]', '', regex=True).astype(int)
plan_df['user_id'] = traveler_ids

# 날짜 변환 - to_date 대신 to_datetime 사용
plan_df['start_date'] = pd.to_datetime(df_plan['TRAVEL_START_YMD'])
plan_df['end_date'] = pd.to_datetime(df_plan['TRAVEL_END_YMD'])

# 추가 필드 복사
plan_df['title'] = df_plan['TRAVEL_NM']
plan_df['purpose'] = df_plan['TRAVEL_PURPOSE']
# location 필드 추가
plan_df['location'] = df_plan['TRAVEL_STATUS_DESTINATION']

print("\nPlan 테이블 미리보기:")
print(plan_df.head())
print(f"Plan 테이블 길이: {len(plan_df)}")

# 결과 CSV 파일로 저장
print("\n데이터 저장 중...")
user_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/user.csv', index=False)
plan_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/plan.csv', index=False)
print("저장 완료!") 

Master 데이터 컬럼: ['TRAVELER_ID', 'RESIDENCE_SGG_CD', 'GENDER', 'AGE_GRP', 'EDU_NM', 'EDU_FNSH_SE', 'MARR_STTS', 'FAMILY_MEMB', 'JOB_NM', 'JOB_ETC', 'INCOME', 'HOUSE_INCOME', 'TRAVEL_TERM', 'TRAVEL_NUM', 'TRAVEL_LIKE_SIDO_1', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SIDO_2', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SIDO_3', 'TRAVEL_LIKE_SGG_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_STATUS_YMD', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_COMPANIONS_NUM']
Travel 데이터 컬럼: ['TRAVEL_ID', 'TRAVEL_NM', 'TRAVELER_ID', 'TRAVEL_PURPOSE', 'TRAVEL_START_YMD', 'TRAVEL_END_YMD', 'MVMN_NM', 'TRAVEL_PERSONA', 'TRAVEL_MISSION', 'TRAVEL_MISSION_CHECK']
Master 데이터 길이: 2560
Travel 데이터 길이: 2560
문자 제거 후: 0    004720
1    000914
2    003564
3    000396
4    001890
Name: TRAVELER_ID, dtype: object
정수 변환 후: 0    4720
1  

## User Profile Data Generation

In [3]:
"""
이 스크립트는 사용자 프로필 데이터를 생성하는 과정입니다.

1. 기존에 생성한 user.csv 파일을 읽어옵니다.
2. 각 사용자에 대한 프로필 정보를 생성합니다:
   - profile_id: 순차적으로 할당 (실제 DB에서는 auto_increment)
   - user_id: user.csv의 user_id와 매핑
   - nickname: user의 name 값 사용
   - 기타 필드: 초기값 설정
3. 생성된 프로필 데이터를 profile.csv 파일로 저장합니다.

이 데이터는 사용자 프로필 관리 및 추천 시스템의 user_preferences 테이블 구성에 활용됩니다.
"""

import pandas as pd
from datetime import datetime

# 기존에 생성한 user_migrated.csv 파일 읽기
user_df = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/user.csv', encoding='utf-8')
profile_df = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/profile.csv', encoding='utf-8')

# 현재 시각을 created_at 값으로 사용 (문자열 형태로 변환)
current_ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# profile 테이블에 맞게 매핑: 
# profile_id: auto_increment 효과를 위해 1부터 순차적으로 할당 (실제 DB에서는 auto_increment 되므로 CSV에서 참고용)
# user_id: user_migrated.csv의 user_id 사용
# nickname: user의 name 컬럼 사용
# profile_image, introduction, updated_at: null 처리 (여기서는 빈 문자열로 표시)
profile_df = pd.DataFrame()
profile_df['profile_id'] = range(1, 1 + len(user_df))
profile_df['user_id'] = user_df['user_id']
profile_df['nickname'] = user_df['name']
profile_df['profile_image'] = None   # 또는 빈 문자열: ''
profile_df['introduction'] = None      # 또는 빈 문자열: ''
profile_df['created_at'] = current_ts
profile_df['updated_at'] = None        # 또는 빈 문자열: ''

# 결과 미리보기
print(profile_df.head())

# profile.csv 파일로 저장
profile_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/profile.csv', index=False, encoding='utf-8')


   profile_id  user_id nickname profile_image introduction  \
0           1        4  e000004          None         None   
1           2        6  e000006          None         None   
2           3        9  e000009          None         None   
3           4       10  e000010          None         None   
4           5       11  e000011          None         None   

            created_at updated_at  
0  2025-03-19 21:05:51       None  
1  2025-03-19 21:05:51       None  
2  2025-03-19 21:05:51       None  
3  2025-03-19 21:05:51       None  
4  2025-03-19 21:05:51       None  


## Place Data Generation

In [ ]:
"""
이 스크립트는 Aihub에서 제공하는 'tn_visit_area_info_방문지정보_E.csv' 데이터를
우리 시스템의 plan.csv와 연결하여, 모든 TRAVEL_ID에 대한 place 테이블 형식 CSV를 생성합니다.

특히 TRAVEL_ID('e_e000004') → user_id(4)처럼,
TRAVEL_ID에서 'e_e000' 접두사를 제거한 뒤 int로 변환하여 user_id를 구한 후,
해당 user_id를 plan.csv와 매핑(plan_id)하는 로직을 포함합니다.

구체적으로 다음 작업을 합니다:
1) plan.csv 로드 (user_id, plan_id 정보를 갖고 있어야 함)
2) 방문지정보 파일 로드 (tn_visit_area_info_방문지정보_E.csv)
3) TRAVEL_ID에서 접두사 제거 후 int 변환 → user_id
4) plan.csv와 user_id를 기준으로 merge하여 plan_id 할당
5) place 테이블 컬럼(place, is_public, visit_date, lat, lon, p_name, p_order 등) 생성
6) 결과를 place_migrated.csv에 저장
"""

import pandas as pd
import numpy as np

# 1) plan.csv 로드
df_plan = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/plan.csv', encoding='utf-8')
print("=== plan.csv 컬럼 ===")
print(df_plan.columns.tolist())

# 여기서 df_plan에는 최소한 'user_id', 'plan_id'가 있다고 가정
# 예) user_id=4, plan_id=1000

# 2) 방문지정보 파일 로드
df_v = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/capital/tn_visit_area_info_방문지정보_E.csv', encoding='utf-8')
print("=== 방문지정보 컬럼 ===")
print(df_v.columns.tolist())

# 3) TRAVEL_ID -> user_id 변환
#    예: 'e_e000004' -> '000004' -> 4, 'f_f000007' -> '000007' -> 7
def convert_travel_id_to_user_id(travel_id: str) -> int:
    # 1) 'e_e' 제거
    # 2) 남은 문자열을 int로 변환
    #    travel_id 예: 'e_e000004' -> re.sub(r'^[a-z]_[a-z]', '', 'e_e000004') => '000004' -> int('000004') => 4
    import re
    cleaned_id = re.sub(r'^[a-z]_[a-z]', '', travel_id)
    return int(cleaned_id)

# 새 컬럼 user_id를 만들어서 TRAVEL_ID를 변환
df_v['user_id'] = df_v['TRAVEL_ID'].apply(convert_travel_id_to_user_id)

# 4) plan.csv와 user_id를 기준으로 merge → plan_id 할당
#    plan.csv에는 user_id가 있고, 이를 통해 plan_id를 가져옴
df_plan_map = df_plan[['user_id', 'plan_id']].drop_duplicates()
df_merged = pd.merge(df_v, df_plan_map, on='user_id', how='inner')

print("=== 병합 결과 샘플 ===")
print(df_merged.head())

# 5) place 테이블용 DataFrame 생성
#    place.csv가 이미 존재한다면, 그 구조를 불러올 수도 있고,
#    여기서는 빈 데이터프레임에 직접 컬럼을 만들어 사용
place_df = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/place.csv', encoding='utf-8')

# place_id : 임시로 1부터 부여 (실제 DB에서는 AUTO_INCREMENT)
place_df['place_id'] = range(1, 1 + len(df_merged))

# plan_id : df_merged['plan_id']
place_df['plan_id'] = df_merged['plan_id']

# place : 도로명 주소(ROAD_NM_ADDR)가 있으면 사용, 없으면 지번주소(LOTNO_ADDR)
place_df['place'] = np.where(
    df_merged['ROAD_NM_ADDR'].notnull() & (df_merged['ROAD_NM_ADDR'] != ''),
    df_merged['ROAD_NM_ADDR'],
    df_merged['LOTNO_ADDR']
)

# is_public : 전부 1로 설정
place_df['is_public'] = 1

# visit_date : VISIT_START_YMD
place_df['visit_date'] = df_merged['VISIT_START_YMD']

# place_cost, memo : null(None)로 처리
place_df['place_cost'] = None
place_df['memo'] = None

# lat, lon : Y_COORD(위도), X_COORD(경도)
place_df['lat'] = df_merged['Y_COORD']
place_df['lon'] = df_merged['X_COORD']

# p_name : 방문지명(VISIT_AREA_NM)
place_df['p_name'] = df_merged['VISIT_AREA_NM']

# p_order : VISIT_ORDER (정수 변환)
place_df['p_order'] = df_merged['VISIT_ORDER'].astype(int)

# 6) 결과 저장
place_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/place.csv', index=False, encoding='utf-8')
print("=== place.csv 파일로 저장 완료 ===")

# 샘플 확인
print(place_df.head(10))


=== plan.csv 컬럼 ===
['plan_id', 'user_id', 'start_date', 'end_date', 'title', 'transport_info', 'location', 'group_id', 'is_public', 'total_cost']
=== 방문지정보 컬럼 ===
['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD', 'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD', 'POI_ID', 'POI_NM', 'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'SGG_CD']
=== 병합 결과 샘플 ===
   VISIT_AREA_ID  TRAVEL_ID  VISIT_ORDER        VISIT_AREA_NM VISIT_START_YMD  \
0     2304300001  e_e000004            1                    집      2023-04-30   
1     2304300002  e_e000004            2  화성 관광열차 안내소 연무대 매표소      2023-04-30   
2     2304300003  e_e000004            3                  창룡문      2023-04-30   
3     2304300004  e_e000004            4            수원 화성 화홍문      2023-04-30   
4     2304300005  e_e000004            5                

In [4]:
'''
p_name에 집 관련 키워드가 포함된 데이터의 공개여부 False로 변경
'''

import pandas as pd

# 1) place.csv 로드
place_df = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/place.csv', encoding='utf-8')

# 2) p_name에 집 관련 키워드가 포함된 데이터 찾기
home_keywords = [
    '광주 집', '딸 집 방문', '광명사거리역 근처 친구네 집', '망포동 527-9 집', '집 강남점', '동행자 집에서 여행 종료', '집 근처', '외할머니 집', '어머니 집', '고모집', '남약주 거주 친구네 집', '할머니 집', '집 도착', '가족 집', '본가 집 도착', '친구네 집','친구 집', '지인 집', '친구 친지 집', '친지 집', '부모님 집', 
    '연남 고을 집', '우리 집', '친지 댁', '여자친구 집', '여자친구 할머니 집',
    '할머니네 집', '지인 집 방문', '본인 집', '친가 집', '동생 집', 
    '친척 집', '엄마 집', '형 집', '고향 집'
]

# 모든 키워드에 대한 조건 생성
home_places = place_df['p_name'].str.contains('|'.join(home_keywords), case=False, na=False)

# 3) 공개여부 변경 - 집 관련 키워드가 포함된 데이터만 0으로 변경
place_df.loc[home_places, 'is_public'] = 0
place_df.loc[place_df['p_name'] == '집', 'is_public'] = 0

# 변경된 데이터 확인
print("변경된 데이터 샘플:")
print(place_df[home_places].head(10) if any(home_places) else "변경된 데이터 없음")

# 4) 결과 저장
place_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/place.csv', index=False, encoding='utf-8')
print(f"=== 집 관련 키워드({len(home_keywords)}개)가 포함된 데이터 공개여부 변경 및 저장 완료 ===")
print(f"총 {home_places.sum()}개 레코드가 비공개로 변경되었습니다.")

변경된 데이터 샘플:
     place_id  plan_id           place  is_public  visit_date  place_cost  \
16         17     1002            푸르지오          0  2023-04-29         NaN   
37         38     1003             NaN          0  2023-04-29         NaN   
96         97     1009            아이파크          0  2023-04-29         NaN   
97         98     3554            아이파크          0  2023-04-29         NaN   
155       156     1016      가온누리빌 101동          0  2023-04-30         NaN   
344       345     1032             상1동          0  2023-05-27         NaN   
367       368     1033       상1동 392번지          0  2023-05-31         NaN   
369       370     1033  경기도 부천시 중동1180          0  2023-06-01         NaN   
380       381     1033      상 1동 392번지          0  2023-06-01         NaN   
382       383     1033          중동1180          0  2023-06-02         NaN   

     memo  lat  lon    p_name  p_order  
16    NaN  NaN  NaN   친구 친지 집        1  
37    NaN  NaN  NaN      친구 집       10  
96    NaN  NaN  N

## Log Data Generation

In [5]:
"""
이 스크립트는 plan.csv, place.csv, 그리고 AIHub의 visit_area_info.csv(방문지정보) 파일을
함께 활용하여 'log.csv'(가상의 여행 기록 테이블)을 만들어내는 예시입니다.

[목표]
1) plan.csv + place.csv를 merge해서 (user_id, plan_id, p_name, visit_date 등) 정보를 확보
2) place.csv + visit_area_info.csv를 merge해서 AIHub의 추가 정보(DGSTFN, REVISIT_INTENTION 등)를 가져옴
3) (1)과 (2)의 병합 결과에서 log 테이블에 필요한 컬럼(log_id, user_id, plan_id, comment, create_at, is_public 등)을 생성
4) 최종 결과를 log.csv 로 저장
"""

import pandas as pd
import numpy as np

# 1) CSV 파일 로드
df_plan = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/plan.csv', encoding='utf-8')
df_place = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/place.csv', encoding='utf-8')
df_visit = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/capital/tn_visit_area_info_방문지정보_E.csv', encoding='utf-8')  # AIHub 방문지정보

print("=== plan.csv ===", df_plan.columns.tolist())
print("=== place.csv ===", df_place.columns.tolist())
print("=== visit_area_info.csv ===", df_visit.columns.tolist())

# 디버깅을 위한 정보 출력
print(f"plan.csv 행 수: {len(df_plan)}")
print(f"place.csv 행 수: {len(df_place)}")
print(f"visit_area_info.csv 행 수: {len(df_visit)}")

# ------------------------------------------------------------------------------
# 2) plan + place 병합
#    place 테이블에는 plan_id가 있고, plan 테이블에는 (plan_id, user_id, start_date, end_date 등)이 있음
#    -> 하나의 DF로 합쳐서 (user_id, plan_id, p_name, place, visit_date, ...) 등을 가진다
# ------------------------------------------------------------------------------
df_plan_place = pd.merge(
    df_place,
    df_plan[['plan_id', 'user_id', 'start_date', 'end_date']],
    on='plan_id',
    how='left'
)

# df_plan_place에는 user_id(플랜 작성자), p_name(방문지명), visit_date 등 정보가 포함됨
print("=== df_plan_place 샘플 ===")
print(df_plan_place.head())
print(f"df_plan_place 행 수: {len(df_plan_place)}")
print(f"df_plan_place NaN 값 확인: \n{df_plan_place.isna().sum()}")

# ------------------------------------------------------------------------------
# 3-1) 디버깅: VISIT_AREA_NM과 p_name 값 확인
# ------------------------------------------------------------------------------
print("=== p_name 고유값 10개 샘플 ===")
print(df_plan_place['p_name'].dropna().unique()[:10])

print("=== VISIT_AREA_NM 고유값 10개 샘플 ===")
print(df_visit['VISIT_AREA_NM'].dropna().unique()[:10])

# ------------------------------------------------------------------------------
# 3-2) place + visit_area_info 병합 (직접 매핑 없이 일단 place.csv만 사용)
# ------------------------------------------------------------------------------
# 일단 AIHub의 방문지정보를 참고하지 않고, place.csv의 데이터만 사용해서 log 생성
df_plan_place_ai = df_plan_place.copy()

# 방문지 만족도와 재방문 의향 생성
import random
df_plan_place_ai['DGSTFN'] = df_visit['DGSTFN']
df_plan_place_ai['REVISIT_INTENTION'] = df_visit['REVISIT_INTENTION']

# ------------------------------------------------------------------------------
# 4) log 테이블용 DataFrame 구성
#    -> 요구사항: place.csv의 is_public이 0인 데이터는 log를 만들지 않고, 
#       생성된 log는 isPublic만 0으로 설정하고 나머지는 null로 설정
# ------------------------------------------------------------------------------
# 먼저 is_public이 1인 장소만 필터링 (공개된 장소만) - 원본 데이터에서 확인
place_public_count = df_plan_place_ai[df_plan_place_ai['is_public'] == 1].shape[0]
place_private_count = df_plan_place_ai[df_plan_place_ai['is_public'] == 0].shape[0]
print(f"공개 장소 수: {place_public_count}, 비공개 장소 수: {place_private_count}")

# 공개용 로그만 생성
log_df = pd.DataFrame()

# log_id 생성: 모든 장소 데이터에 대해 일련번호 부여
log_df['log_id'] = range(1, 1 + len(df_plan_place_ai))

# is_public: 모든 데이터에 대해 place의 is_public 값 그대로 사용
log_df['is_public'] = df_plan_place_ai['is_public']

# place의 is_public 값에 따라 선택적으로 데이터 채우기
# is_public = 1인 장소 데이터만 내용 채우기
# is_public = 0인 장소 데이터는 NaN으로 두기
is_public_mask = df_plan_place_ai['is_public'] == 1

# user_id, plan_id, place_id 채우기 - 공개 데이터만
log_df['user_id'] = np.where(is_public_mask, df_plan_place_ai['user_id'], np.nan)
log_df['plan_id'] = np.where(is_public_mask, df_plan_place_ai['plan_id'], np.nan)
log_df['place_id'] = np.where(is_public_mask, df_plan_place_ai['place_id'], np.nan)

# NaN 값이 포함되어 있으면 int로 변환할 수 없으므로, 저장 전에 필요한 부분만 int로 변환
# 공개 데이터에 대해서만 정수형으로 변환
mask = log_df['is_public'] == 1
if mask.any():
    log_df.loc[mask, 'user_id'] = log_df.loc[mask, 'user_id'].astype(int)
    log_df.loc[mask, 'plan_id'] = log_df.loc[mask, 'plan_id'].astype(int)
    log_df.loc[mask, 'place_id'] = log_df.loc[mask, 'place_id'].astype(int)

# comment 생성 - 공개 데이터만
def make_comment(row):
    if row['is_public'] != 1:
        return np.nan
        
    p_name = row.get('p_name', '알 수 없는 장소')
    dgs = row.get('DGSTFN', np.nan)
    rvt = row.get('REVISIT_INTENTION', np.nan)
    
    # None값을 방지하기 위해 문자열 변환:
    if pd.isna(dgs):
        dgs = 'N/A'
    if pd.isna(rvt):
        rvt = 'N/A'
    
    if pd.isna(p_name) or p_name == '':
        p_name = '알 수 없는 장소'
    
    return f"[{p_name}] 만족도={dgs}, 재방문={rvt}"

log_df['comment'] = df_plan_place_ai.apply(make_comment, axis=1)

# create_at: 공개 데이터만 날짜 사용, 비공개는 NaN
log_df['create_at'] = np.where(is_public_mask, df_plan_place_ai['visit_date'], np.nan)


# NaN 값 확인
print("=== log_df NaN 값 확인 ===")
print(log_df.isna().sum())
print(f"총 레코드: {len(log_df)}, 공개(is_public=1): {log_df['is_public'].sum()}, 비공개(is_public=0): {len(log_df) - log_df['is_public'].sum()}")

# ------------------------------------------------------------------------------
# 5) 결과 저장
# ------------------------------------------------------------------------------
log_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/log.csv', index=False, encoding='utf-8')
print("=== log.csv 생성 완료 ===")
# 공개/비공개 데이터 각각 5개씩 출력
print("\n=== 공개 데이터 샘플 ===")
print(log_df[log_df['is_public'] == 1].head(5))
print("\n=== 비공개 데이터 샘플 ===")
print(log_df[log_df['is_public'] == 0].head(5)) 

=== plan.csv === ['plan_id', 'user_id', 'start_date', 'end_date', 'title', 'transport_info', 'location', 'group_id', 'is_public', 'total_cost']
=== place.csv === ['place_id', 'plan_id', 'place', 'is_public', 'visit_date', 'place_cost', 'memo', 'lat', 'lon', 'p_name', 'p_order']
=== visit_area_info.csv === ['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD', 'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD', 'POI_ID', 'POI_NM', 'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'SGG_CD']
plan.csv 행 수: 2560
place.csv 행 수: 21495
visit_area_info.csv 행 수: 21384
=== df_plan_place 샘플 ===
   place_id  plan_id                    place  is_public  visit_date  \
0         1     1000                      NaN          0  2023-04-30   
1         2     1000  경기 수원시 팔달구 창룡대로103번길 20          1  2023-04-30   
2         3     1000   

## Follow Data Generation

In [10]:
"""
user.csv를 읽어 가상의 follow.csv를 생성한다.
 - 비슷한 지역/스타일을 선호하면 팔로우 확률을 높임
 - 일부 사용자를 '인플루언서'로 지정해 많이 팔로우 받음
"""

import pandas as pd
import numpy as np

# --------------------
# 하이퍼 파라미터
# --------------------
AVG_FOLLOW_OUT = 5          # 한 사용자가 팔로우하는 평균 수
INFLU_RATIO    = 0.05       # 인플루언서 비율 (상위 5%)
W_SIM          = 0.7        # 유사 취향 가중치
W_RAND         = 0.3        # 무작위 가중치
SEED           = 42
np.random.seed(SEED)

# 1) 사용자 로드
user_df = pd.read_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/user.csv')  # 최소 user_id 컬럼 필요
user_ids = user_df['user_id'].tolist()
N = len(user_ids)

# 2) 유사도 행렬(0~1) 만들기 ─ 여기서는
#    같은 region / 스타일이면 1, 아니면 0 의 단순 예시
#    (선호 데이터가 없다면 전부 0 배열로 두고 완전 랜덤)
sim = np.zeros((N, N))

if {'pref_region','pref_style'}.issubset(user_df.columns):
    for i, u in enumerate(user_ids):
        for j, v in enumerate(user_ids):
            if i==j:
                continue
            same_region = user_df.at[i, 'pref_region'] == user_df.at[j, 'pref_region']
            same_style  = user_df.at[i, 'pref_style']  == user_df.at[j, 'pref_style']
            sim[i, j] = 0.5*same_region + 0.5*same_style  # 둘다 같으면 1, 하나만 같으면 0.5

# 3) 인플루언서 선정 (팔로워 많이 받음)
infl_n = max(1, int(N * INFLU_RATIO))
infl_ids = np.random.choice(user_ids, infl_n, replace=False)

# 4) 팔로우 생성
follows = []
f_id = 1
for idx, uid in enumerate(user_ids):
    # 팔로우할 수 candidate pool (자기 제외)
    candidates = [x for x in user_ids if x != uid]

    # 기본 팔로우 수 = Poisson(AVG_FOLLOW_OUT)
    k = max(1, np.random.poisson(lam=AVG_FOLLOW_OUT))

    # 가중치:  w_sim * sim + w_rand * uniform
    weights = (W_SIM * sim[idx] + W_RAND * np.random.rand(N))
    weights[idx] = 0  # self 0
    # 인플루언서는 weight 보정(많이 팔로우 받도록)
    for j, cand in enumerate(user_ids):
        if cand in infl_ids:
            weights[j] *= 2.5   # 가중치 상승

    # 정규화 후 샘플
    probs = weights / weights.sum()
    to_follow = np.random.choice(user_ids, size=k, replace=False, p=probs)

    for target in to_follow:
        follows.append({'f_id': f_id, 'from_user': uid, 'to_user': int(target)})
        f_id += 1

# 5) deduplicate 혹시 모를 중복 제거
follow_df = pd.DataFrame(follows).drop_duplicates()
follow_df.to_csv('/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our/follow_migrated.csv', index=False)
print(f"생성된 follow 수: {len(follow_df)} / 사용자 수: {N}")


생성된 follow 수: 12682 / 사용자 수: 2560


## Like Data Generation

In [9]:
import pandas as pd
import numpy as np
rng = np.random.default_rng(seed=42)

# 데이터 로드
log_df     = pd.read_csv("/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/log.csv")
follow_df  = pd.read_csv("/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/follow.csv")          # from_user, to_user
visit_df   = pd.read_csv("/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/capital/tn_visit_area_info_방문지정보_E.csv") # DGSTFN 등

# 1) 작성자 → DGSTFN 매핑 대비용
log_meta = log_df[['log_id', 'user_id', 'create_at']].merge(
    visit_df[['VISIT_AREA_ID', 'DGSTFN']],
    left_on='log_id',      # log_id == VISIT_AREA_ID 라고 가정(이미 매핑)
    right_on='VISIT_AREA_ID',
    how='left'
)

likes = []

for _, row in log_meta.iterrows():
    author = row['user_id']
    likers = follow_df.loc[follow_df['to_user']==author, 'from_user'].tolist()   # 팔로워들
    candidate_pool = set(likers) | set(np.random.choice(log_df['user_id'], 5))   # 팔로워+랜덤 5명
    
    for u in candidate_pool:
        # 동일 사용자가 자기 글엔 좋아요 안 누르게 할 수도 있음
        if u == author: 
            continue
        
        # 확률 결정
        if u in likers:
            p = 0.35
        elif row['DGSTFN']>=4:
            p = 0.20
        else:
            p = 0.05
        
        if rng.random() < p:
            likes.append({'log_id': row['log_id'], 'user_id': u})

like_df = pd.DataFrame(likes).drop_duplicates()
like_df.to_csv("/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/like.csv", index=False)

## Log Comment (댓글) Data Generation

In [11]:
templates = {
  5: ["와, 최고네요!", "여기 꼭 가봐야겠어요😍", "사진이 정말 멋져요!"],
  4: ["좋아보여요!", "꿀팁 감사해요", "가보고 싶어요"],
  3: ["정보 고마워요!", "재밌어 보입니다"],
  2: ["아쉬웠군요😢", "다음엔 더 좋길!"],
  1: ["헉… 별로였나 봐요", "정보 공유 감사"]
}

import itertools, datetime as dt
comments = []
loco_id  = itertools.count(1)

for _, row in log_meta.iterrows():
    author   = row['user_id']
    rating   = int(row['DGSTFN']) if not np.isnan(row['DGSTFN']) else 3
    pool_temp= templates.get(rating, templates[3])

    followers = follow_df.loc[follow_df['to_user']==author, 'from_user']
    cand_non  = rng.choice(log_df['user_id'], 5, replace=False)
    for u in set(followers) | set(cand_non):
        if u==author:
            continue
        p = 0.12 if u in followers.values else 0.03
        if rng.random() < p:
            # create_at 처리: NaT면 현재시간을 기본값으로 사용
            base_time = pd.to_datetime(row['create_at'])
            if pd.isnull(base_time):
                base_time = pd.Timestamp.now()
            comment_time = base_time + pd.Timedelta(minutes=int(rng.integers(5, 720)))
            comments.append({
                'loco_id'    : next(loco_id),
                'log_id'     : row['log_id'],
                'user_id'    : u,
                'loco_comment': rng.choice(pool_temp),
                'create_at'  : comment_time.strftime('%Y-%m-%d %H:%M:%S'),
                'parent_id'  : None
            })

comment_df = pd.DataFrame(comments)
comment_df.to_csv("/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/log_comment.csv", index=False)


## User Action Data Generation

In [12]:
ua = []
ts_now = pd.Timestamp('now')

# ① post
ua.extend([{
    'action_id'  : i,
    'user_id'    : r['user_id'],
    'target_id'  : r['log_id'],
    'action_type': 'post',
    'target_type': 'log',
    'action_time': r['create_at']
} for i, r in log_df.iterrows()])

# ② like
for _, r in like_df.iterrows():
    ua.append({
        'action_id'  : None,
        'user_id'    : r['user_id'],
        'target_id'  : r['log_id'],
        'action_type': 'like',
        'target_type': 'log',
        'action_time': ts_now.isoformat()
    })

# ③ comment
for _, r in comment_df.iterrows():
    ua.append({
        'action_id'  : None,
        'user_id'    : r['user_id'],
        'target_id'  : r['log_id'],
        'action_type': 'comment',
        'target_type': 'log',
        'action_time': r['create_at']
    })

user_actions_df = pd.DataFrame(ua)
user_actions_df['action_id'] = range(1, len(user_actions_df)+1)
user_actions_df.to_csv("/Users/kimmo/TRAVEL-ON-NA/TravelOnNa_AI/Data/our_c/user_actions.csv", index=False)
